# Experiments

In [ ]:
import io

import ipywidgets as widgets
import pandas as pd

pd.set_option('display.float_format', '{0:,.0f}'.format)

In [ ]:
# *** constants ***

# All measurements expressed in meters unless noted
BREAKOFF_ALTITUDE = 1707.0
EXIT_SPEED = 9.81/2
MAX_SPEED_ACCURACY = 3.0
PERFORMANCE_WINDOW_LENGTH = 2256.0
VALIDATION_WINDOW_LENGTH = 1006.0

In [ ]:
uploader = widgets.FileUpload(description = 'Speed CSV', tooltip = 'FlySight speed file')
display(uploader)

In [ ]:
data = pd.read_csv(io.BytesIO(uploader.value[0].content), skiprows= (1,1))

## Use meaningful column names

In [ ]:
data['heightFt'] = data['hMSL'].apply(lambda h: 3.2808*h)
data['timeUnix'] = data['time'].apply(lambda t: pd.Timestamp(t).timestamp())
data = pd.DataFrame(data = {
    'timeUnix': data.timeUnix,
    'altitudeMSL': data.hMSL,
    'heightFt': data.heightFt,
    'vMetersPerSecond': data.velD,
    'vKMh': data.velD*3.6,
    'speedAccuracy': data.sAcc, })

## Discard non-actionable rows

Discard all rows before maximum altitude...

In [ ]:
timeMaxAlt = data[data.altitudeMSL == data.altitudeMSL.max()].timeUnix.iloc[0]
data = data[data.timeUnix > timeMaxAlt]

Then discard all rows where height < 0; appears to be a bug in FlySight MSL altitude handling.

In [ ]:
data = data[data.altitudeMSL > 0]

## Find the freefall data subset

In [ ]:
def dataGroups(data):
    data_ = data.copy()
    data_['positive'] = (data_.vMetersPerSecond > 0)
    data_['group'] = (data_.positive != data_.positive.shift(1)).astype(int).cumsum()-1

    return data_

In [ ]:
data = dataGroups(data)
groups = data.group.max()+1
print('Data groups = %d' % groups)

freeFallGroup = -1
dataPoints = -1
for group in range(groups):
    subset = data[data.group == group]
    if len(subset) > dataPoints:
        freeFallGroup = group
        dataPoints = len(subset)
display(freeFallGroup)

data = data[data.group == freeFallGroup]
data = data.drop('group', axis = 1).drop('positive', axis = 1)

### Drop data before exit and below the breakoff altitude

Breakoff altitude is 1,707 meters - FAI ISC and USPA Competition rules.

In [ ]:
data = data[data.vMetersPerSecond > EXIT_SPEED]
data = data[data.altitudeMSL >= BREAKOFF_ALTITUDE]

### Identify performance, scoring, and validation window

The PERFORMANCE_WINDOW_LENGTH is 2,256 meters after exit
If the performance window is below the breakoff altitude, scoring ends at BREAKOFF_ALTITUDE

In [ ]:
windowStart = data.iloc[0].altitudeMSL
windowEnd = windowStart-PERFORMANCE_WINDOW_LENGTH
if windowEnd < BREAKOFF_ALTITUDE:
    windowEnd = BREAKOFF_ALTITUDE

validationWindowStart = windowEnd+VALIDATION_WINDOW_LENGTH
data = data[data.altitudeMSL >= windowEnd]

print('Window start = {0:,.2f}'.format(windowStart))
print('Validation window start = {0:,.2f}'.format(validationWindowStart))
print('Window end = {0:,.2f}'.format(windowEnd))

---
## Jump validation
Every data sample within the validation window must satisfy the precision criterium of max speed accuracy < 3 m/s; 0.0 == most accurate.

In [ ]:
speedAccuracy = data[data.altitudeMSL < validationWindowStart].speedAccuracy.max()

if speedAccuracy < MAX_SPEED_ACCURACY:
    color = '#0f0'
    result = '🟢 valid'
else:
    color = '#f00'
    result = '🔴 invalid'

validJumpStatus = '<hr><h1><span style="color: %s">%s jump</span></h1>' % (color, result)

---
## Jump analysis

In [ ]:
table = None

for column in pd.Series([ 5.0, 10.0, 15.0, 20.0, 25.0, ]):
    timeOffset = data.iloc[0].timeUnix+column
    tranche = data.query('timeUnix == %f' % timeOffset).copy()
    tranche['time'] = [ column, ]

    if pd.isna(tranche.iloc[-1].vKMh):
        tranche = data.tail(1).copy()
        tranche['time'] = tranche.timeUnix-data.iloc[0].timeUnix
    
    if table is not None:
        table = pd.concat([ table, tranche, ])
    else:
        table = tranche

table = pd.DataFrame([ table.time, table.vKMh, ])

In [ ]:
display(widgets.HTML(validJumpStatus))
display(table)
display(widgets.HTML('<h3>Max speed = {0:,.0f}</h3>'.format(data.vKMh.max())))